In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
import torch.nn as nn
import wandb
import torch
random_seed = 42

In [159]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [160]:
X_train.shape

(12713, 245)

In [161]:
class MyTensorDataset(torch.utils.data.Dataset):
    def __init__(self, *args, **kwargs):
        assert len(args) > 0
        len_base = len(args[0])
        
        for i in args:
            assert len(i) == len_base
        
        self.data = args
        
    def __len__(self, ):
        return len(self.data[0])
        
    def __getitem__(self, idx):
        return [t[idx] for t in self.data]

In [162]:
train_dataset = MyTensorDataset(torch.FloatTensor(X_train.values), torch.LongTensor(Y_train.values))
test_dataset = MyTensorDataset(torch.FloatTensor(X_test.values), torch.LongTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,)

In [163]:
net = nn.Sequential(nn.Linear(245, 2), nn.Linear(2, 1), nn.Sigmoid())
n_epoch = 20
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.NLLLoss()

In [164]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        print(x_batch)
        y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        print(y_batch)
        # Forward
        predictions = net(x_batch)
        print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print('Current loss: %f' % loss)

tensor([[7.2200e+02, 2.2000e+01, 7.0000e+00,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.5600e+03, 2.2000e+01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         3.0000e+00],
        [8.9300e+02, 2.2000e+01, 7.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [2.7400e+03, 2.2000e+01, 7.0000e+00,  ..., 0.0000e+00, 1.4000e+01,
         5.0000e+00],
        [7.5800e+02, 2.2000e+01, 7.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.3900e+03, 2.2000e+01, 7.0000e+00,  ..., 0.0000e+00, 1.0000e+00,
         2.0000e+00]])
tensor([0, 1, 1, 0, 1, 0, 1, 0])
tensor([[0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [4.1334e-11],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00]], grad_fn=<SigmoidBackward0>)


IndexError: Target 1 is out of bounds.

In [ ]:
test_preds = np.array([])

for x_batch, y_batch in test_dataloader:
    predictions = net(x_batch)
    print(predictions)
    test_preds = np.append(test_preds, predictions.argmax(dim=-1).numpy())

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [

In [ ]:
test_preds 

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5


In [ ]:
ut.write(model)